In [1]:
import gymnasium as gym
import numpy as np
from tqdm import tqdm
from typing import NamedTuple

In [2]:
class Params(NamedTuple):
    total_episodes: int  # Total episodes
    n_runs: int  # Number of runs
    epsilon: float  # Exploration probability
    epsilon_decay: float  # Decay epsilon
    epsilon_min: float  # Epsilon minimum value
    alpha: float  # Learning rate
    gamma: float  # Discounting rate
    action_size: int  # Number of possible actions
    state_size: int  # Number of possible states
    seed: int  # Define a seed so that we get reproducible results

params = Params(
    total_episodes=500,
    n_runs=3,
    alpha=0.5,
    gamma=0.95,
    epsilon=1,
    epsilon_decay=0.005,
    epsilon_min=0.01,
    action_size=None,
    state_size=None,
    seed=123,
)

# Set the seed
rng = np.random.default_rng(params.seed)

In [3]:
env = gym.make('Taxi-v3', render_mode="rgb_array")
# env.metadata['render_fps']=120

params = params._replace(action_size=env.action_space.n)
params = params._replace(state_size=env.observation_space.n)
print(f"Action size: {params.action_size}")
print(f"State size: {params.state_size}")

Action size: 6
State size: 500


In [4]:
class Qlearning:
    def __init__(self, alpha, gamma, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.alpha = alpha
        self.gamma = gamma
        self.reset_qtable()

    def update(self, state, action, reward, new_state):
        """Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]"""
        delta = (
            reward
            + self.gamma * np.max(self.qtable[new_state, :])
            - self.qtable[state, action]
        )
        q_update = self.qtable[state, action] + self.alpha * delta
        return q_update

    def reset_qtable(self):
        """Reset the Q-table."""
        self.qtable = np.zeros((self.state_size, self.action_size))


class EpsilonGreedy:
    def __init__(self, epsilon, epsilon_min, epsilon_decay):
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

    def choose_action(self, action_space, state, qtable):
        """Choose an action `a` in the current world state (s)."""
        # First we randomize a number
        explor_exploit_tradeoff = rng.uniform(0, 1)
        # Exploration
        if explor_exploit_tradeoff < self.epsilon:
            action = action_space.sample()
            decision_type = "exploration"
        # Exploitation (taking the biggest Q-value for this state)
        else:
            # Break ties randomly
            # Find the indices where the Q-value equals the maximum value
            # Choose a random action from the indices where the Q-value is maximum
            max_ids = np.where(qtable[state, :] == max(qtable[state, :]))[0]
            action = rng.choice(max_ids)
            decision_type = "exploitation"
        return action, decision_type

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon - self.epsilon_decay, self.epsilon_min)

class Tester:
    def pick_action(self, state, qtable):
        max_ids = np.where(qtable[state, :] == max(qtable[state, :]))[0]
        action = rng.choice(max_ids)
        return action

In [5]:
learner = Qlearning(
    alpha=params.alpha,
    gamma=params.gamma,
    state_size=params.state_size,
    action_size=params.action_size,
)


def run_env():
    rewards = np.zeros((params.total_episodes, params.n_runs))
    steps = np.zeros((params.total_episodes, params.n_runs))
    wins = np.zeros((params.total_episodes, params.n_runs))
    truncated_episodes = np.zeros((params.total_episodes, params.n_runs))
    explorations = np.zeros(params.n_runs)
    exploitations = np.zeros(params.n_runs)

    episodes = np.arange(params.total_episodes)
    qtables = np.zeros((params.n_runs, params.state_size, params.action_size))
    all_states = []
    all_actions = []

    for run in range(params.n_runs): # Run several times to account for stochasticity
        learner.reset_qtable()  # Reset the Q-table between runs

        explorer = EpsilonGreedy(
            epsilon=params.epsilon,
            epsilon_decay=params.epsilon_decay,
            epsilon_min=params.epsilon_min,
        )

        for episode in tqdm(
            episodes, desc=f"Run {run}/{params.n_runs} - Episodes", leave=False
        ):

            state = env.reset(seed=params.seed)[0]
            done = False
            total_rewards = 0 # +20 delivering passenger else -10 illegal action else -1
            total_steps = 0

            while not done:
                action, decision_type = explorer.choose_action(
                    action_space=env.action_space, state=state, qtable=learner.qtable
                )

                explorations[run] += 1 if decision_type == "exploration" else 0
                exploitations[run] += 1 if decision_type == "exploitation" else 0

                # Log all states and actions
                all_states.append(state)
                all_actions.append(action)

                # Take the action (a) and observe the outcome state(s') and reward (r)
                new_state, reward, terminated, truncated, info = env.step(action)

                if action == 5:
                    print('Dropped passager')
                    print("terminated :" + str(terminated))
                    print("truncated :" + str(truncated))

                done = terminated or truncated

                learner.qtable[state, action] = learner.update(
                    state, action, reward, new_state
                )

                total_steps += 1
                total_rewards += reward

                # Our new state is state
                state = new_state

            truncated_episodes[episode, run] = 1 if truncated else 0
            wins[episode, run] = 1 if terminated and reward == 20 else 0

            # Log all rewards and steps
            rewards[episode, run] = total_rewards
            steps[episode, run] = total_steps

            explorer.decay_epsilon()

        qtables[run, :, :] = learner.qtable

    return rewards, steps, episodes, qtables, all_states, all_actions, wins, explorations, exploitations, truncated_episodes

rewards, steps, episodes, qtables, all_states, all_actions, wins, explorations, exploitations, truncated_episodes = run_env()

Run 0/3 - Episodes:   0%|          | 0/500 [00:00<?, ?it/s]

Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped pass

Run 0/3 - Episodes:   6%|▋         | 32/500 [00:00<00:01, 315.90it/s]

Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passa

Run 0/3 - Episodes:  24%|██▍       | 120/500 [00:00<00:00, 644.42it/s]

...

Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped pass

Run 2/3 - Episodes:  19%|█▉        | 97/500 [00:00<00:00, 519.36it/s]

Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :True
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passa

Run 2/3 - Episodes:  66%|██████▌   | 329/500 [00:00<00:00, 1339.61it/s]


terminated :True
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :False
truncated :False
Dropped passager
terminated :True
truncated :False
Dropped passager
terminated :True
trunc